In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow as tf

In [3]:
data = tfds.load(name="imdb_reviews", split=["train[:60%]", "train[60%:]", "test"], as_supervised=True)

In [4]:
train_data,val_data,test_data=data

In [5]:
x=train_data.batch(100)

In [6]:
x_inputs,x_labels=next(iter(x))

In [7]:
print(x_labels)

tf.Tensor(
[0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1
 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 0 1 0 0 0 1 0
 1 1 1 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 1 1 1 0 1], shape=(100,), dtype=int64)


In [8]:
print(x_inputs[0])

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)


In [9]:
# Load a different pre-trained embedding model from TensorFlow Hub
url = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1"
hub = hub.KerasLayer(url, input_shape=[], trainable=True, dtype=tf.string)

In [10]:
model = tf.keras.Sequential([
    hub,
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 64)                3264      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 48193929 (183.85 MB)
Trainable params: 48193929 (183.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

In [14]:
history = model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=20,
    validation_data=val_data.batch(512),
    verbose=1,
    callbacks=[early_stopping]  # Adding early stopping callback
)

Epoch 1/20
30/30 [==============================] - 18s 560ms/step - loss: 0.7641 - accuracy: 0.5068 - val_loss: 0.7430 - val_accuracy: 0.5407
Epoch 2/20
30/30 [==============================] - 16s 528ms/step - loss: 0.7479 - accuracy: 0.5347 - val_loss: 0.7325 - val_accuracy: 0.6017
Epoch 3/20
30/30 [==============================] - 16s 526ms/step - loss: 0.7361 - accuracy: 0.5591 - val_loss: 0.7215 - val_accuracy: 0.6480
Epoch 4/20
30/30 [==============================] - 16s 522ms/step - loss: 0.7243 - accuracy: 0.5838 - val_loss: 0.7093 - val_accuracy: 0.6767
Epoch 5/20
30/30 [==============================] - 16s 520ms/step - loss: 0.7109 - accuracy: 0.6149 - val_loss: 0.6962 - val_accuracy: 0.6969
Epoch 6/20
30/30 [==============================] - 16s 535ms/step - loss: 0.6973 - accuracy: 0.6419 - val_loss: 0.6829 - val_accuracy: 0.7097
Epoch 7/20
30/30 [==============================] - 16s 523ms/step - loss: 0.6817 - accuracy: 0.6699 - val_loss: 0.6691 - val_accuracy: 0.7210

In [15]:
results = model.evaluate(test_data.batch(512), verbose=0)

In [16]:
for name, value in zip(model.metrics_names, results):
    print("%s: %.5f" % (name, value))

loss: 0.49519
accuracy: 0.81116
